# Кузнецов Михаил Пи19-4


# Антагонистические игры



In [174]:
import pandas as pd
import numpy as np
import fractions

## Численный метод Брауна-Робинсон

In [175]:
class Game:
    def __init__(self, game_matrix: np.array, mixed_1: np.array, mixed_2: np.array, mixed_cost_1: float, mixed_cost_2: float):
        self.game_matrix = game_matrix
        self.mixed_1 = mixed_1
        self.mixed_2 = mixed_2
        self.mixed_cost_1 = mixed_cost_1
        self.mixed_cost_2 = mixed_cost_2

        self.pure_1 = 0
        self.pure_cost_1 = - np.inf
        n, m = game_matrix.shape
        for i in range(n):
            tmp = np.min(game_matrix[i, : ])
            if tmp > self.pure_cost_1:
                self.pure_1 = i
                self.pure_cost_1 = tmp
 
        self.pure_2 = 0
        self.pure_cost_2 = np.inf
        
        for i in range(m):
            tmp = np.max(game_matrix[ : ,i ])
            if tmp < self.pure_cost_2:
                self.pure_2 = i
                self.pure_cost_2 = tmp

    def __str__(self):
        return "".join([
        "mixed strat of А: {}\n".format(self.mixed_1),
        "mixed strat of Б: {}\n".format(self.mixed_2),
        "pure strat of А: {}\n".format(self.pure_1),
        "pure strat of Б: {}\n".format(self.pure_2),
        "mixed cost of А: {}\n".format(self.mixed_cost_1),
        "mixed cost of Б: {}\n".format(self.mixed_cost_2),
        "pure cost of А: {}\n".format(self.pure_cost_1),
        "pure cost of Б: {}\n".format(self.pure_cost_2)],
        )

class BrownRobinsonMethod:
    def __init__(self, eps = 10**-3):
        self.eps = eps

    def solve(self, game_matrix: np.array):
        m, n = game_matrix.shape

        # Вектора смешанных стратегий
        x = np.zeros(m)
        y = np.zeros(n)

        # Вектора выигрыша 1 игрока и проигрыша 2 игрока
        win_1 = np.zeros(m)
        loss_2 = np.zeros(n)

        # Текущие стратегии 1 и 2 игроков; в начале можно и случайный выбор
        cur_st_1 = 0 # np.random.randint(m)
        cur_st_2 = 0 # np.random.randint(n)

        lower_bounds = []
        upper_bounds = []


        step = 0
        cur_eps = np.inf
        while cur_eps > self.eps:
            # Каждый из игроков исходит от выбора другого игрока на шаге
            # Игрок 1 среди выбора стратегии игрока 2 пытается найти наиболее выигрышную стратегию
            # Игрок 2 среди выбора стратегии игрока 1 пытается найти наименее проигрышную стратегию
            step += 1

            win_1 = np.add(win_1, game_matrix[ : ,cur_st_2])
            loss_2 = np.add(loss_2, game_matrix[cur_st_1, : ])

            x[cur_st_1] += 1
            y[cur_st_2] += 1

            lower_bound = np.divide(np.min(loss_2), step)
            upper_bound = np.divide(np.max(win_1), np.double(step))
  
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)

            cur_eps = np.min(upper_bounds) - np.max(lower_bounds)

            cur_st_1 = np.argmax(win_1)
            cur_st_2 = np.argmin(loss_2)


        cost_1 = max(lower_bounds) + np.divide(cur_eps, 2)
        cost_2 = min(upper_bounds) - np.divide(cur_eps, 2)

        # Собственно, считаем, с какой вероятностью выбираем каждую стратегию
        x = np.divide(x, step)
        y = np.divide(y, step)

        # Чистая - наиболее вероятная в векторе смешанных
        return Game(game_matrix, x, y, cost_1, cost_2)

## Тестовый пример

In [176]:
example = np.array([
  [300, 280, 230],
  [200, 180, 130],
  [240, 380, 400]
])

brm = BrownRobinsonMethod(10**-1)
sol = brm.solve(example)

strats_a = ["Телефоны", "Ноутбуки", "ПК"]
strats_b = ["Телефоны", "Наушники", "Ноутбуки"]

print(sol)

mixed strat of А: [0.69465649 0.         0.30534351]
mixed strat of Б: [0.80916031 0.         0.19083969]
pure strat of А: 2
pure strat of Б: 0
mixed cost of А: 281.7251113231552
mixed cost of Б: 281.7251113231552
pure cost of А: 240
pure cost of Б: 300



### Смешанные стратегии игроков

In [177]:
df1 = pd.DataFrame(data = [sol.mixed_1], columns = strats_a)
df1

,Телефоны,Ноутбуки,ПК
0,0.694656,0.0,0.305344


In [178]:
df2 = pd.DataFrame(data = [sol.mixed_2], columns = strats_b)
df2

,Телефоны,Наушники,Ноутбуки
0,0.80916,0.0,0.19084


In [179]:
print("Цена игры А при выборе смешанной стратегии: {}".format(sol.mixed_cost_1))
print("Цена игры Б при выборе смешанной стратегии: {}".format(sol.mixed_cost_2))

Цена игры А при выборе смешанной стратегии: 281.7251113231552
Цена игры Б при выборе смешанной стратегии: 281.7251113231552


### Чистые оптимальные стратегии игроков

In [180]:
print("Чистая стратегия игрока А: {}, стоимость игры: {}".format(sol.pure_1, sol.pure_cost_1))
print("Чистая стратегия игрока Б: {}, стоимость игры: {}".format(sol.pure_2, sol.pure_cost_2))

Чистая стратегия игрока А: 2, стоимость игры: 240
Чистая стратегия игрока Б: 0, стоимость игры: 300


In [181]:
class GameData:
    def __init__(self):
        self.n = 0
        self.m = 0
        self.strats_a = None
        self.strats_b = None
        self.matrix = None
        self.solution = None

    def play(self):
        brm = BrownRobinsonMethod(10**-1)
        self.solution = brm.solve(self.matrix)

    def print_results(self):
        print("\nРезультаты решения:\n")
        print("Чистая стратегия игрока А: {} - {}, стоимость игры: {}".format(self.solution.pure_1 + 1, self.strats_a[self.solution.pure_1], self.solution.pure_cost_1))
        print("Чистая стратегия игрока Б: {} - {}, стоимость игры: {}".format(self.solution.pure_2 + 1, self.strats_b[self.solution.pure_2], self.solution.pure_cost_2))

        df1 = pd.DataFrame(data = [self.solution.mixed_1], columns = self.strats_a)
        print("Смешанная стратегия игрока А:\n{}".format(df1))
        print("Цена игры А при выборе смешанной стратегии: {}".format(self.solution.mixed_cost_1))

        df2 = pd.DataFrame(data = [self.solution.mixed_2], columns = self.strats_b)
        print("Смешанная стратегия игрока Б:\n{}".format(df2))
        print("Цена игры Б при выборе смешанной стратегии: {}".format(self.solution.mixed_cost_2))



#### Метод для ввода пользователем матрицы весовых коэффициентов и названия стратегий с клавиатуры

In [182]:
def enterCoef(n, m):
    game_matrix = [ [0]*m for i in range(n) ]
    names_of_strategies_a = [0] * n
    names_of_strategies_b = [0] * m
    # ввод стратегий игрока А
    for i in range(n):
        name = input(f"Введите название  {i+1} стратегии игрока А: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_a[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
     # ввод стратегий игрока Б
    for i in range(m):
        name = input(f"Введите название  {i+1} стратегии игрока Б: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_b[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
            
    for i in range(n):
        for j in range(m):
            num = int(input(f"Введите элемент строки {i+1} столбца {j+1}: "))
            if num > 0:
                game_matrix[i][j] = num
            else:
                raise ValueError
            
    return (np.array(game_matrix), names_of_strategies_a, names_of_strategies_b)

#### Метод для ввода пользователем названия стратегий с клавиатуры и случайная генерация матрицы весовых коэффициентов

In [183]:
def randomCoef(n,m):
    names_of_strategies_a = [0] * n
    names_of_strategies_b = [0] * m
    
    for i in range(n):
        name = input(f"Введите название  {i+1} стратегии игрока А: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_a[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
            
    for i in range(m):
        name = input(f"Введите название  {i+1} стратегии игрока Б: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_b[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError 
            
    game_matrix = np.random.randint(1, 500, size = (n,m))
    
    return (game_matrix, names_of_strategies_a, names_of_strategies_b)

#### Метод для считывания из csv

In [184]:
def readFromScv(path = "data/data.csv"):
    main_matrix = pd.read_csv(path)
    names_of_strategies_a = list(main_matrix['Names_a'])
    main_matrix.drop('Names_a', inplace=True, axis=1)
    game_matrix = None
    names_of_strategies_b = list(main_matrix.columns)
    if sum(main_matrix.isnull().sum()):
        print("Содержатся пропуски!")
        raise ValueError
    else:
        if sum(main_matrix.dtypes == object) > 0:
            print("Содержится строка в файле!")
            raise ValueError
        elif (main_matrix <= 0).any().any() == True:
            print("Содержится отрицательное или нулевое значение!")
            raise ValueError
        else:
            game_matrix = main_matrix.to_numpy()
            
    return (game_matrix, names_of_strategies_a, names_of_strategies_b)

In [191]:
games = {1:"Ввод с клавиатуры", 2:"Случайная генерация", 3:"Чтение из csv"}

print(f"Выберите один из 3 вариантов получения данных \n Доступные варианты получения данных: {games}")
print()

game_matrix = None
game_names = None

try: 
    choice = int(input("Введите число для получения данных (от 1 до 3): "))
    
    if (choice > 3) or (choice <= 0):
        print("Введите число от 1 до 3!")
    
    else:
        print(f"Выбранный вариант получения данных: {games[choice]}")
        
        if choice == 1:
            try:
                n = int(input("Введите кол-во стратегий для игрока А: "))
                m = int(input("Введите кол-во стратегий для игрока Б: "))
                
                if (n > 0) and (m > 0):
                    game_matrix, names_of_strategies_a, names_of_strategies_b  = enterCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                 print("Прервано пользователем")
        elif choice == 2:
            try:
                n = int(input("Введите кол-во стратегий для игрока А: "))
                m = int(input("Введите кол-во стратегий для игрока Б: "))
                
                if (n > 0) and (m > 0):
                    game_matrix, names_of_strategies_a, names_of_strategies_b  = randomCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
        else:
            try:
                game_matrix, names_of_strategies_a, names_of_strategies_b = readFromScv()
            except FileNotFoundError:
                print("Файл не найден!")
            except ValueError:
                print("Измените значения ячеек в файле!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
            
except ValueError:
    print("Неправильный ввод!")
except KeyboardInterrupt:
    print("Прервано пользователем")

Выберите один из 3 вариантов получения данных 
 Доступные варианты получения данных: {1: 'Ввод с клавиатуры', 2: 'Случайная генерация', 3: 'Чтение из csv'}

Введите число для получения данных (от 1 до 3): 3
Выбранный вариант получения данных: Чтение из csv
   Telephones  Notebooks  Tablets  Apples  Potatos
0         300        280      230     300      500
1         200        180      130     500      900
2         240        380      400     600       12


### Пример ввода

In [192]:
game = GameData()
game.matrix = game_matrix
game.strats_a = names_of_strategies_a
game.strats_b = names_of_strategies_b
print(game.matrix)
print(game.strats_a)
print(game.strats_b)
game.play()
game.print_results()

[[300 280 230 300 500]
 [200 180 130 500 900]
 [240 380 400 600  12]]
['Tels', 'PC', 'Phones']
['Telephones', 'Notebooks', 'Tablets', 'Apples', 'Potatos']

Результаты решения:

Чистая стратегия игрока А: 1 - Tels, стоимость игры: 230
Чистая стратегия игрока Б: 1 - Telephones, стоимость игры: 300
Смешанная стратегия игрока А:
       Tels        PC    Phones
0  0.694832  0.000765  0.304403
Цена игры А при выборе смешанной стратегии: 281.70859206133764
Смешанная стратегия игрока Б:
   Telephones  Notebooks  Tablets  Apples   Potatos
0    0.746855        0.0  0.25289     0.0  0.000255
Цена игры Б при выборе смешанной стратегии: 281.70859206133764
